In [ ]:
# prompt: create a optimiaztion problem that revolves around a shop that needs to be open for 81 hours in  a week. There are 4 empolyees that have coresponding time avability and hourly wage: Ania can work 30 hours from monday to friday for 35$/hour, Barbara which can work any number of hours all week for 100$/hour, Krzysztof can work  20 hours saturday and sunday for 35$/hour, Franek can work all week for 50 hours for 50$/hour. The restrictions are that they need to be paid for full hours and hours are intigers. Use the pulp library.

!pip install pulp ## Instalacja biblioteki Pulp do rozwiązywania problemów optymalizacyjnych

from pulp import * # Importowanie wszystkich funkcji z biblioteki PuLP

problem = LpProblem("Employee Scheduling", LpMinimize) # Tworzenie problemu optymalizacyjnego z celem minimalizacji kosztów zatrudnienia

# Definiowanie zmiennych decyzyjnych określających liczbę godzin pracy dla każdego pracownika
# Każda zmienna ma określone ograniczenia minimalne i maksymalne
ania_hours = LpVariable("Ania_Hours", 0, 30, LpInteger)  # Ania może pracować maksymalnie 30 godzin
barbara_hours = LpVariable("Barbara_Hours", 0, None, LpInteger) # Barbara nie ma ograniczenia maksymalnego
krzysztof_hours = LpVariable("Krzysztof_Hours", 0, 20, LpInteger) # Krzysztof może pracować maksymalnie 20 godzin
franek_hours = LpVariable("Franek_Hours", 0, 50, LpInteger) # Franek może pracować maksymalnie 50 godzin


# Funkcja celu: minimalizacja całkowitego kosztu zatrudnienia
problem += 35 * ania_hours + 100 * barbara_hours + 35 * krzysztof_hours + 50 * franek_hours, "Total Cost"

# Ograniczenie: Suma godzin pracy wszystkich pracowników musi wynosić dokładnie 81 godzin
problem += ania_hours + barbara_hours + krzysztof_hours + franek_hours == 81, "Total Hours"

# Rozwiązanie problemu optymalizacyjnego
problem.solve()

# Wyświetlenie statusu rozwiązania
print("Status:", LpStatus[problem.status])

# Wyświetlenie optymalnej liczby godzin pracy dla każdego pracownika
print("Ania Hours:", ania_hours.varValue)
print("Barbara Hours:", barbara_hours.varValue)
print("Krzysztof Hours:", krzysztof_hours.varValue)
print("Franek Hours:", franek_hours.varValue)

# Wyświetlenie minimalnego kosztu zatrudnienia wynikającego z rozwiązania
print("Total Cost: $", value(problem.objective))

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.7/17.7 MB 44.9 MB/s eta 0:00:00
Status: Optimal
Ania Hours: 30.0
Barbara Hours: 0.0
Krzysztof Hours: 20.0
Franek Hours: 31.0
Total Cost: $ 3300.0


/usr/local/lib/python3.10/dist-packages/pulp/pulp.py:1298: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


In [ ]:
# prompt: create a optimiaztion problem that revolves around a shop that needs to be open for 81 hours in  a week. There are 4 empolyees that have coresponding time avability and hourly wage: Ania can work 30 hours  for 35$/hour, Barbara which can work any number of hours for 100$/hour, Krzysztof can work  20  hours for 35$/hour, Franek 50 hours for 50$/hour. The restrictions are that they need to be paid for full hours, hours are intigers, and there are 24 hours in a day and each person can work only on specific days of the week ( Ania on Monday, Tuesday, Wednesday, Thursday, Friday, Barbara can work all week, Krzysztof can work on Saturday, Sunday, Franek can work all week  . List all days and print who will work on each day and for how many hours. The shop needs to be open every day. Each person can work only for 8 hours in a day . Use the pulp library

from pulp import *

# Tworzenie problemu optymalizacyjnego z celem minimalizacji kosztów zatrudnienia
problem = LpProblem("Employee Scheduling", LpMinimize)

# Definiowanie pracowników
employees = ["Ania", "Barbara", "Krzysztof", "Franek"]

# Definiowanie dni tygodnia
days = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]

# Tworzenie zmiennych decyzyjnych określających liczbę godzin pracy dla każdego pracownika w danym dniu
# Każda zmienna ma dolne ograniczenie 0 i górne 8 (maksymalnie 8 godzin pracy dziennie)
hours = LpVariable.dicts("Hours", (employees, days), lowBound=0, upBound=8, cat='Integer')


# Maksymalna liczba godzin pracy w tygodniu dla każdego pracownika
employee_hours = {
    "Ania": 30,  # Ania może pracować maksymalnie 30 godzin w tygodniu
    "Barbara": None,   # Barbara nie ma ograniczenia godzinowego
    "Krzysztof": 20,  # Krzysztof może pracować maksymalnie 20 godzin w tygodniu
    "Franek": 50,  # Franek może pracować maksymalnie 50 godzin w tygodniu
}

# Koszt godzinowy pracy każdego pracownika
hourly_wages = {
    "Ania": 35,
    "Barbara": 100,
    "Krzysztof": 35,
    "Franek": 50,
}

# Dostępność pracowników w poszczególne dni tygodnia
availability = {
    "Ania": ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday"], # Ania pracuje tylko w dni robocze
    "Barbara": days, # Barbara może pracować każdego dnia
    "Krzysztof": ["Saturday", "Sunday"], # Krzysztof pracuje tylko w weekendy
    "Franek": days # Franek może pracować każdego dnia
}

# Funkcja celu: minimalizacja całkowitego kosztu zatrudnienia
problem += lpSum([hourly_wages[employee] * hours[employee][day] for employee in employees for day in days]), "Total Cost"

# 1. Ograniczenie maksymalnej liczby godzin pracy dla każdego pracownika w tygodniu
for employee in employees:
  if employee_hours[employee] is not None:
    problem += lpSum([hours[employee][day] for day in days if day in availability[employee]]) <= employee_hours[employee], f"{employee}_Total_Hours"

# 2. Całkowita liczba godzin do przepracowania w tygodniu (81 godzin)
problem += lpSum([hours[employee][day] for employee in employees for day in days if day in availability[employee]]) == 81, "Total_Shop_Hours"

# 3. Sklep musi być otwarty każdego dnia (musi być przynajmniej jeden pracownik dziennie)
for day in days:
  problem += lpSum([hours[employee][day] for employee in employees if day in availability[employee]]) >= 1 , f"{day}_Open"

# 4. Pracownicy mogą pracować tylko w dni, w które są dostępni
for employee in employees:
    for day in days:
        if day not in availability[employee]: # Jeśli dany pracownik nie jest dostępny danego dnia
            problem += hours[employee][day] == 0, f"{employee}_Unavailable_{day}"

# Rozwiązanie problemu optymalizacyjnego
problem.solve()

# Wyświetlenie statusu rozwiązania (np. "Optimal" jeśli udało się znaleźć optymalne rozwiązanie)
print("Status:", LpStatus[problem.status])

# Wyświetlenie minimalnego kosztu zatrudnienia wynikającego z rozwiązania
print("Total Cost: $", value(problem.objective))


for day in days:
    print(f"\n{day}:")
    daily_hours = 0  # Suma godzin pracy danego dnia
    for employee in employees:
        if hours[employee][day].varValue > 0:  # Sprawdzenie, czy dany pracownik ma przydzielone godziny
            print(f"  {employee}: {hours[employee][day].varValue} hours")
            daily_hours += hours[employee][day].varValue # Sumowanie godzin pracy danego dnia
    print(f"Total hours for {day} : {daily_hours}")  # Wyświetlenie sumy godzin dla danego dnia

Status: Optimal
Total Cost: $ 3360.0

Monday:
  Ania: 8.0 hours
  Franek: 8.0 hours
Total hours for Monday : 16.0

Tuesday:
  Ania: 8.0 hours
Total hours for Tuesday : 8.0

Wednesday:
  Ania: 8.0 hours
Total hours for Wednesday : 8.0

Thursday:
  Ania: 6.0 hours
  Franek: 8.0 hours
Total hours for Thursday : 14.0

Friday:
  Franek: 8.0 hours
Total hours for Friday : 8.0

Saturday:
  Krzysztof: 8.0 hours
  Franek: 8.0 hours
Total hours for Saturday : 16.0

Sunday:
  Krzysztof: 8.0 hours
  Franek: 3.0 hours
Total hours for Sunday : 11.0
